# ELT

Data Engineering Week 9 <br>
ETL/ELT 실습

#### 간단한 웹 페이지 : http://43.200.129.41:8000/

초기 설정

In [58]:
# 한글말고 영어로 본인의 닉네임을 정해주세요!
name = "seungmin"

In [59]:
# 하단의 링크 주소는 master server의 주소입니다.
url = "http://43.200.129.41"
port = "8000"

request_url = url + ":" + port

In [60]:
### Log 확인
import requests, json
response = requests.get(request_url + "/" + "logs", params = {"caller_name": name})
logs = json.dumps(response.json(), indent=2, ensure_ascii=False)

# print(logs)

In [61]:
### User DB 확인
response = requests.get(request_url + "/" + "users", params={"caller_name": name})
users = json.dumps(response.json(), indent=2, ensure_ascii=False)

# print(users)

In [62]:
logs_object = json.loads(logs)
users_object = json.loads(users)

In [63]:
from datetime import datetime, timezone, timedelta

kst = timezone(timedelta(hours=9))

for i in range (len(logs_object)) :
    s = logs_object[i]["timestamp"].replace("Z", "+00:00")
    dt = datetime.fromisoformat(s)
    logs_object[i]["timestamp"] = dt.astimezone(kst).replace(microsecond=0).isoformat()

In [101]:
# 임의로 분리
past_log = logs_object[-350:-1]

# Week 7
### 기존에 쌓인 log를 처리하는 것

다음과 같은 Task를 수행했습니다.
- Task1. log 확인
- Task2. 각 사용자의 마지막 접속 시간
- Task3. 각 사용자의 행동 패턴
- Task4. 각 포스트의 조회수, 하트 수 확인
- Task5. 각 날짜 별 누가 접속했는지

In [102]:
import datetime

# Task 2. 각 사용자의 마지막 접속 시간
def last_login (logs) -> list[int] :

    # 각 index = user_id라 생각하고,
    # user_login[1]은 user_id가 1인 사용자의 마지막 login 시간이 저장되도록
    user_login = [0] * (len(users_object) + 1)

    for l in logs:
        if l.get("action") != "login":
            continue

        uid = l["user_id"]
        ts  = l["timestamp"]

        if user_login[uid] == 0 or ts > user_login[uid]:
            user_login[uid] = ts

    # index가 0 인 사용자는 없으니까 list의 첫번째 값이 0인 것은 정상
    return user_login


# Task 4. 각 포스트의 조회수, 하트 수 확인
def update_post (logs, post_id : int) -> list[int] :

    view = 0
    like = 0

    for log in logs:
        if log.get("post_id") != post_id:
            continue

        event = log.get("action")

        if event == "post_click" or event == "view_post":
            view += 1
        elif event == "like" or event == "like_post":
            like += 1


    return [view, like]

In [103]:
# 모든 post의 값을 얻어오는 코드 (수정 불필요)
def update_all_post (logs, post_num = 6) :

    data = []

    for i in range (1, post_num + 1) :
        data.append(update_post(logs, i))

    return data

In [104]:
# 작성했던 코드를 동작시키기 위한 job_code (수정 불필요)
def ELT_job (logs) :

    # log 처리
    data = {}

    data["last_login"] = last_login(logs)               # 7-2 / 8-2
    data["update_all_post"] = update_all_post(logs)     # 7-4 / 8-3


    data["last_updated"] = logs[0]["timestamp"]

    return data

In [105]:
past_data = ELT_job(past_log)

In [106]:
print(past_data)

{'last_login': [0, '2025-11-04T19:06:26+09:00', '2025-11-04T11:13:15+09:00', '2025-11-04T16:26:32+09:00', '2025-11-11T18:59:18+09:00', '2025-11-04T16:51:49+09:00', '2025-11-11T19:24:05+09:00', '2025-11-04T18:39:52+09:00', '2025-11-04T19:36:13+09:00', '2025-11-04T18:40:17+09:00', '2025-11-04T18:40:19+09:00', '2025-11-11T18:31:34+09:00', '2025-11-11T18:32:20+09:00'], 'update_all_post': [[4, 3], [8, 5], [7, 5], [3, 6], [9, 9], [14, 22]], 'last_updated': '2025-11-14T09:41:47+09:00'}


이미 쌓인 로그(과거)데이터를 정상적으로 처리하여 저장했습니다.

---

# Week 8
### 이어서 log를 처리하는 것

다음과 같은 Task를 수행했습니다.
- Task. log 분리

## Task
현재 로그를 받아보면, 이미 처리한 데이터를 포함하고 있습니다. <br>
(이미 처리한 데이터 이후 ~ 새로 들어온 데이터)로 분리하세요. <br>


In [107]:
def seperate_log (logs) :
    last_timestamp = past_log[0]["timestamp"]

    logs = [log for log in logs if log["timestamp"] > last_timestamp]

    after_log = logs

    return after_log

In [108]:
current_log = seperate_log(logs_object)
print(current_log)

[{'id': 407, 'user_id': 6, 'post_id': None, 'action': 'page_view', 'details': 'Viewed page: home', 'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36', 'timestamp': '2025-11-18T18:52:37+09:00'}, {'id': 406, 'user_id': 6, 'post_id': 4, 'action': 'like', 'details': 'Liked post 4', 'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36', 'timestamp': '2025-11-18T18:52:36+09:00'}, {'id': 405, 'user_id': 6, 'post_id': 4, 'action': 'like', 'details': 'Liked post 4', 'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36', 'timestamp': '2025-11-18T18:52:35+09:00'}, {'id': 404, 'user_id': 6, 'post_id': 2, 'action': 'like', 'details': 'Liked post 2', 'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36', '

이로써 과거에 처리한 데이터부터, 최신 데이터까지 log를 분리했습니다.

---

# Week 9
### 과거 이미 처리된 데이터와 방금 처리한 데이터 통합

다음과 같은 Task를 수행합니다.
- Task. 분리된 log 기반으로, 데이터를 처리하고, 과거 데이터와 통합

## Task
이미 처리된 past_data와 방금 처리한 present_data를 어떻게 통합할 것인가?

In [109]:
present_data = ELT_job(current_log)
print(present_data)

{'last_login': [0, '2025-11-18T15:33:35+09:00', 0, 0, 0, 0, '2025-11-18T18:50:33+09:00', '2025-11-18T14:32:37+09:00', 0, 0, 0, 0, 0], 'update_all_post': [[0, 1], [0, 2], [0, 1], [0, 2], [1, 1], [0, 2]], 'last_updated': '2025-11-18T18:52:37+09:00'}


In [110]:
def Merge (past_data, present_data) :

    # 기존 처리된 data와 방금 처리한 데이터 통합
    data = {}

    user_cnt = len(past_data["last_login"])  # 같은 사용자 수
    data["last_login"] = [0] * user_cnt

    for i in range(user_cnt):
        if present_data["last_login"][i] == 0:
            data["last_login"][i] = past_data["last_login"][i]
        else:
            data["last_login"][i] = present_data["last_login"][i]

    row = len(past_data["update_all_post"])
    col = len(past_data["update_all_post"][0])

    data["update_all_post"] = [[past_data["update_all_post"][i][j] + present_data["update_all_post"][i][j]for j in range(col)]for i in range(row)]


    data["last_updated"] = present_data["last_updated"] # 수정 X

    return data

In [111]:
result = Merge(past_data, present_data)
print(result)

{'last_login': [0, '2025-11-18T15:33:35+09:00', '2025-11-04T11:13:15+09:00', '2025-11-04T16:26:32+09:00', '2025-11-11T18:59:18+09:00', '2025-11-04T16:51:49+09:00', '2025-11-18T18:50:33+09:00', '2025-11-18T14:32:37+09:00', '2025-11-04T19:36:13+09:00', '2025-11-04T18:40:17+09:00', '2025-11-04T18:40:19+09:00', '2025-11-11T18:31:34+09:00', '2025-11-11T18:32:20+09:00'], 'update_all_post': [[4, 4], [8, 7], [7, 6], [3, 8], [10, 10], [14, 24]], 'last_updated': '2025-11-18T18:52:37+09:00'}


# 과제 완료
모두 수고하셨습니다! <br>
완성하신 파일을 카톡으로 보내주시고, 실행한 결과 값(result)도 함께 보내주시면 됩니다! <br>